<a href="https://colab.research.google.com/github/Sakinat-Folorunso/OOU_DTS201_Introduction_to_Data_Science/blob/main/notebooks/DTS201_Week9_PH_Student_Centred.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DTS 201 – INTRODUCTION TO DATA SCIENCE  
## WEEK 9 PRACTICAL (PH): BIG DATA, LARGE FILES & WORKING IN JUPYTER / GOOGLE COLAB

**Instructor:** DR SAKINAT FOLORRUNSO – ASSOCIATE PROFESSOR OF AI SYSTEMS AND FAIR DATA  
**Department:** COMPUTER SCIENCES, OLABISI ONABANJO UNIVERSITY, AGO-IWOYE, OGUN STATE, NIGERIA  

---

### 🎯 Practical Learning Objectives (Week 9)

By the end of this practical session, you should be able to:

- Explain the basic idea of **Big Data** and why large files are challenging.  
- Describe key features of **Jupyter Notebooks** and **Google Colab** as data science tools.  
- Simulate a **large dataset** and inspect its memory usage.  
- Load and process data using **chunking** (`chunksize`) instead of reading everything at once.  
- Apply simple analytics (e.g. computing totals and averages) on large files in a **memory-efficient** way.  

> 💡 This lab is a gentle introduction to “big data thinking” using **simulated large CSV files** rather than real multi-GB datasets, to keep it safe for student laptops and lab machines.


## 1. What Do We Mean by “Big Data”?

In practice, **Big Data** often means data that is:

- Too **large** to fit into memory (RAM) on a single machine.  
- Too **fast** (high-velocity streams) to process one-by-one in real time without special tools.  
- Too **complex** (variety of formats: text, images, logs, clicks, sensors) to handle with simple spreadsheets.

In this course, we will **not** work with terabytes of data, but we will:

- Think about how **file size** and **memory** affect analysis.  
- Practise techniques like **chunking** and **type optimisation**.  
- Use tools like **Jupyter** and **Google Colab** effectively.


### 1.1 Jupyter Notebooks and Google Colab (Quick Orientation)

Both **Jupyter Notebooks** and **Google Colab** allow you to:

- Write and run code in small blocks called **cells**.  
- Mix **code**, **output**, and **explanatory text** (markdown) in one document.  
- Save, share, and re-run experiments easily.  

Key cell types:

- **Code cells** – contain Python code that you can run.  
- **Markdown cells** – contain formatted text for explanations, comments, and notes.

> ✏️ **Student Task:**  
> - Click on a markdown cell in this notebook and try editing some text.  
> - Run the cell (Shift + Enter) to see how the formatting updates.


### 1.2 Optional: Mount Google Drive (If Using Google Colab)

In [ ]:
# Run this cell ONLY if:
# 1. You are working in Google Colab, AND
# 2. Your data files are stored in your Google Drive.

# Import the drive module from google.colab to allow mounting Google Drive
from google.colab import drive  # Provides tools to connect Colab to your Google Drive

# Mount your Google Drive so that it appears as a folder in Colab
drive.mount('/content/drive')  # Follow the instructions and paste the authorization code when asked

# After mounting, you can access files under '/content/drive/MyDrive/'
# Example path: '/content/drive/MyDrive/Datasets/large_file.csv'

## 2. Importing Required Libraries

In [ ]:
# Import pandas for working with tabular data (rows and columns)
import pandas as pd

# Import numpy for numerical operations and random number generation
import numpy as np

# Import matplotlib and seaborn for simple plots (optional for this lab)
import matplotlib.pyplot as plt
import seaborn as sns

# Ensure plots appear inside the notebook
%matplotlib inline

# Set a visual style for plots
plt.style.use("seaborn-v0_8")

## 3. Simulating a Large Dataset in Memory

To avoid downloading very large files, we will **simulate** a dataset with many rows.

We create a DataFrame with, for example:

- An `id` column (1, 2, 3, …).  
- A `category` column (e.g. A, B, C).  
- A numerical column like `value` (random numbers).  
- A `timestamp` column (simulated dates).  

You can adjust the number of rows to fit your machine.

> ⚠️ **Warning:** Do not make this too large on a weak machine (e.g. avoid 5 million rows on an old laptop).  
> Start modestly (e.g. 100,000 rows) and only increase if your system is responsive.


In [ ]:
# Choose how many rows to simulate
# Start with 100_000 (one hundred thousand). You can try increasing to 500_000 or 1_000_000 later.
n_rows = 100_000  # TODO: adjust based on your machine's capacity

# Set a random seed so results are reproducible
np.random.seed(42)  # Ensures that random numbers are the same each time you run the cell

# Create a sequence of integer IDs from 1 to n_rows
ids = np.arange(1, n_rows + 1)  # Unique identifier for each row

# Create a categorical column with a few repeating categories
categories = np.random.choice(['A', 'B', 'C', 'D'], size=n_rows)  # Randomly assign A/B/C/D

# Create a numerical column of random values (e.g. from a normal distribution)
values = np.random.normal(loc=100, scale=15, size=n_rows)  # Mean 100, standard deviation 15

# Create a column of random dates within a given range
dates = pd.to_datetime('2024-01-01') + pd.to_timedelta(
    np.random.randint(0, 365, size=n_rows), unit='D'
)  # Random dates in 2024

# Combine into a single DataFrame
df_large = pd.DataFrame({
    'id': ids,
    'category': categories,
    'value': values,
    'date': dates
})

# Display the first few rows to confirm the structure
df_large.head()

> ✏️ **Student Task 1:**  
> - Change `n_rows` to a smaller value (e.g. 10_000) and then to a larger value (e.g. 300_000 if your machine can handle it).  
> - Observe whether your notebook becomes slower to respond when working with larger n_rows.  


### 3.1 Checking Memory Usage of the DataFrame

In [ ]:
# Use memory_usage to see how much memory each column consumes
memory_per_column = df_large.memory_usage(deep=True)  # deep=True gives a more accurate measurement for object columns

# Display memory usage per column (in bytes)
memory_per_column

In [ ]:
# Compute total memory usage in megabytes (MB)
total_memory_bytes = memory_per_column.sum()  # Sum memory used by all columns
total_memory_mb = total_memory_bytes / (1024 ** 2)  # Convert bytes to megabytes

print(f"Total memory usage of df_large: {total_memory_mb:.2f} MB")

> ✏️ **Student Reflection 2:**  
> - How much memory (in MB) does your `df_large` use?  
> - What happens to the memory usage if you double `n_rows`? Does it roughly double as well?  


## 4. Saving the Large Dataset to CSV (Simulating a Large File)

To simulate working with a **large file from disk**, we will:

1. Save `df_large` to a CSV file.  
2. Pretend that we are now another analyst loading this file in **chunks** instead of reading everything at once.


In [ ]:
# Specify a file name for saving the large dataset
large_csv_path = "simulated_large_dataset.csv"  # This file will be created in the current working directory

# Save the DataFrame to CSV without the index column
df_large.to_csv(large_csv_path, index=False)  # Writes the entire DataFrame to a CSV file

print(f"Saved large dataset to: {large_csv_path}")

## 5. Loading and Processing the Large CSV File in Chunks

When files become too large to load entirely into memory, we can use **chunking**, where:

- The file is read in **small portions** (e.g. 10,000 rows at a time).  
- Each chunk is processed, and partial results are **aggregated**.  

We will compute the **overall mean** and **sum** of the `value` column using chunks.


In [ ]:
# Choose a chunk size (number of rows per chunk)
chunk_size = 20_000  # Adjust based on your machine; smaller chunks use less memory

# Initialize variables to accumulate results
total_sum = 0.0         # Will hold the sum of 'value' over entire file
total_count = 0         # Will hold the total number of rows

# Use pandas.read_csv with chunksize to iterate over the file
for chunk in pd.read_csv(large_csv_path, chunksize=chunk_size):
    # For each chunk, compute the sum and count of the 'value' column
    chunk_sum = chunk['value'].sum()     # Sum of 'value' in this chunk
    chunk_count = chunk['value'].count() # Number of non-missing entries in this chunk

    # Update total accumulators
    total_sum += chunk_sum
    total_count += chunk_count

# After processing all chunks, compute the overall mean
overall_mean = total_sum / total_count

print("Processed file in chunks of size:", chunk_size)
print("Total rows processed:", total_count)
print("Overall mean of 'value' (computed with chunks):", overall_mean)

In [ ]:
# For comparison, compute the mean directly from df_large in memory
direct_mean = df_large['value'].mean()

print("Direct mean from in-memory DataFrame:", direct_mean)

> ✏️ **Student Task 3:**  
> - Compare the chunk-based overall mean and the direct in-memory mean. Are they (almost) the same?  
> - Change `chunk_size` to a smaller number (e.g. 5_000) and re-run. Did the result change? Did the process become slower or faster?  


## 6. Grouped Aggregation in Chunks (By Category)

We can also perform **grouped aggregation** (e.g. average value per category) while reading the file in chunks.

This is more realistic, as many big data tasks require grouping by category, date, or region.


In [ ]:
# Initialize an empty dictionary to accumulate sums and counts by category
category_sums = {}   # Will map category -> sum of 'value'
category_counts = {} # Will map category -> count of 'value'

# Iterate over the CSV file in chunks again
for chunk in pd.read_csv(large_csv_path, chunksize=chunk_size):
    # Group the chunk by 'category' and compute sum and count for 'value'
    grouped = chunk.groupby('category')['value'].agg(['sum', 'count'])

    # Update the global accumulators for each category
    for cat, row in grouped.iterrows():
        # Get current sum and count for this category from the dictionaries (default 0 if not present)
        category_sums[cat] = category_sums.get(cat, 0.0) + row['sum']
        category_counts[cat] = category_counts.get(cat, 0) + row['count']

# After processing all chunks, compute mean value per category
category_means = {cat: category_sums[cat] / category_counts[cat] for cat in category_sums.keys()}

# Convert the result into a DataFrame for display
category_summary = pd.DataFrame({
    'category': list(category_means.keys()),
    'mean_value': list(category_means.values())
})

# Display the summary
category_summary

> ✏️ **Student Task 4:**  
> - Which category has the highest mean value? Which has the lowest?  
> - How could such grouped statistics be useful in a business or scientific context (e.g. comparing branches, products, or patient groups)?  


## 7. Reducing Memory Usage with Data Type Optimisation

One important technique with large data is to **reduce memory usage** by selecting more efficient data types.

Examples:

- Using `int32` instead of `int64` when values are small.  
- Using `float32` instead of `float64` when full precision is not necessary.  
- Converting repeated text fields to **categorical** type.


In [ ]:
# Show original data types
print("Original dtypes:")
print(df_large.dtypes)

# Create a copy of the DataFrame for optimisation
df_optimized = df_large.copy()

# Convert 'id' from default (int64) to int32 to save memory
df_optimized['id'] = df_optimized['id'].astype('int32')

# Convert 'category' from object (string) to category type
df_optimized['category'] = df_optimized['category'].astype('category')

# Convert 'value' from float64 to float32
df_optimized['value'] = df_optimized['value'].astype('float32')

# Note: 'date' can remain as datetime64[ns]

# Check new data types
print("
Optimized dtypes:")
print(df_optimized.dtypes)

In [ ]:
# Compare memory usage before and after optimisation
original_mem_mb = df_large.memory_usage(deep=True).sum() / (1024 ** 2)
optimized_mem_mb = df_optimized.memory_usage(deep=True).sum() / (1024 ** 2)

print(f"Original memory usage: {original_mem_mb:.2f} MB")
print(f"Optimized memory usage: {optimized_mem_mb:.2f} MB")

> ✏️ **Student Reflection 5:**  
> - How much memory did you save (in MB and as a percentage) after optimisation?  
> - Why is this kind of optimisation important when working with much larger datasets (e.g. millions of rows, many columns)?  


## 8. Mini Big Data Exercise

> 🧪 **Student Activity:**  
> Using the techniques in this notebook, complete the following:

1. Change `n_rows` to simulate a dataset that is **large but still manageable** on your machine.  
2. Save it to CSV and process it in chunks to compute:  
>    - Overall mean and standard deviation of `value`.  
>    - Mean value by category.  
3. Optimise dtypes and report:  
>    - Original memory usage.  
>    - Optimised memory usage.  
4. Write a short paragraph describing:  
>    - What challenges you observed when increasing the dataset size.  
>    - How chunking and type optimisation can help in real big data scenarios.


## 9. Summary and Reflection (Week 9 PH)

In this Week 9 practical, you have:

- Reviewed the concept of **Big Data** and its challenges for memory and computation.  
- Explored the use of **Jupyter/Colab** notebooks for interactive data science.  
- Simulated a **large dataset** and examined its memory usage.  
- Practised reading and processing a large CSV file in **chunks** using `pandas.read_csv(chunksize=...)`.  
- Performed global and grouped aggregations on chunked data.  
- Reduced memory footprint using **data type optimisation**.

> 🧠 **Final Reflection (1–2 paragraphs):**  
> - How do today’s activities change your thinking about working with “big” datasets in practice?  
> - What limitations do Jupyter/Colab still have when dealing with truly massive data (terabytes or more)?  
> - Which of the techniques (chunking, dtypes, simulation) do you think you will use again in future projects?  
